# Introduction à pytorch

In [ ]:
import torch
import torch.nn.functional as F
from torch.autograd import grad
from copy import deepcopy
import matplotlib.pyplot as plt

## I) Les tenseurs

L'outil central de la librairie et la classe torch.Tensor qui permet de faire des calculs vectorizés et être indéxé à la manière d'une array numpy.
C'est un bloc contigue de mémoire qui contient des données typées, et est découpé en plusieurs dimensions. En plus du type de données, on peut définir si le tenseur requiert la propagation du gradient, et l'appareil sur lequel les données sont stockées et où les opérations seront effectuées ("cpu" ou "cuda:i").

In [ ]:
t = torch.tensor([[1, 5], [3, 2]], dtype=torch.float32, requires_grad=False, device="cpu")  # la fonction "torch.tensor" permet de créer un tenseur de manière analogue à "np.array"
print(repr(t))

In [ ]:
2*t - t**0.5 + 2  # Les opérations usuelles en numpy sont disponibles

In [ ]:
(t.shape, t.dtype, t.requires_grad, t.device)  # les attributs usuels des array numpy sont nommées de manière identiques

In [ ]:
t[1, 1], t[:, 1], t[..., 0], t[0], t[:, :1]  # Les syntaxes d'indentation sont identiques à celles de numpy

Toutes les opérations usuelles de la librairie numpy existent dans pytorch (ou presque). Si le tenseur est sur un GPU, le calcul sera effectué de manière parallélisée sur GPU, ce qui peut très largement accélérer la vitesse d'execution pour des tenseurs de grande taille.

In [ ]:
t.min(), t.mean(dim=1), t.max(), t.std(dim=0)  # les méthodes usuelles des array numpy sont nommées de manière identiques. Le kwargs "ax" est renommé "dim".

In [ ]:
t @ -t  # Les opérateur aux sens spécifiques en numpy sont aussi implémentés. Ici équivalent à "torch.matmul(t, -t)".

In [ ]:
torch.exp(-t**2 + 2)  # les fonctions mathématiques courantes sont disponibles dans le namespace "torch"

In [ ]:
F.elu(-t)  # Les fonctions liées à des opérations propres aux réseaux de neurones (loss, convolutions, activation, ...) se trouvent dans le namespace "torch.nn.functional"

In [ ]:
torch.linalg.inv(t)  # Les fonctions d'algèbre linéaires de numpy sont implémentées dans le namespace "torch.linalg". Ici une opération d'inversion matricielle (dont on peut obtenir le gradient !)

In [ ]:
torch.rand((1,)), torch.randint(0, 3, (1,)), torch.normal(0., 1., (1,))  # Les fonctions de génération de nombres aléatoires ne sont pas (contrairement à numpy) dans le namespace torch.random !

Tout comme les arrays numpy, on peut réorganiser la taille des dimensions (sans que la taille totale du tenseur ne change) ou l'ordre des dimensions

In [ ]:
t = torch.arange(2*3*3, dtype=torch.long).reshape(3, 2, 3)
print(repr(t))

In [ ]:
t.transpose(1, 2)  # inverse l'ordre des dimensions 1 et 2

In [ ]:
t.permute(1, 0, 2)  # permute l'ordre de toutes les dimensions dans l'ordre spécifié

In [ ]:
t.unsqueeze(0).shape  # la méthode "unsqueeze" permet de créer des dimensions vide

In [ ]:
t.unsqueeze(0).squeeze(0).shape  # la méthode "squeeze" à l'inverse permet de retirer une dimension vide

## II) création du graph de calcul

Lorsqu'un tenseur requiert la propagation du gradient, le graphe de calcul est créé dynamiquement, de manière transparente pour l'utilisateur, lorsqu'on effectue des opérations, le résultat renvoyé est un nouveau tenseur lié au précédent par une branche du graphe deu calcul. Pour cette raison, il ne faut jamais modifier manuellement le contenu des tenseurs à travers lesquels on compte propager le gradient.

In [ ]:
t = torch.arange(4, dtype=torch.float32, requires_grad=True)
print(repr(t))

In [ ]:
r = torch.sum(t**2)  # notez la présence de la "grad_fn" (gradient function) qui a donné lieu au nouveau tenseur.
print(repr(r))

In [ ]:
grad(r, t, retain_graph=True)  # la fonction "grad" permet de calculer le gradient d'un scalaire par apport à un tenseur donné

In [ ]:
r.backward()  # la méthode "backward" permet de propager le gradient dans tous le graph
print(repr(t.grad))  # on retrouve la solution analytique de la dérivée de la somme des carrés (y = 2*x), évaluées en [0, 1, 2, 3].

In [ ]:
# Une fois le gradient propagé, les résultats intermédiaires du graph de calcul sont supprimés pour libérer de la mémoire.
# On ne peut pas appeler backward une deuxième fois
try:
    r.backward()
except Exception as e:
    print(str(e))

In [ ]:
t = torch.arange(4, dtype=torch.float32, requires_grad=True)
r = torch.sum(t**2)  # appeler "backward" avec le flag "retain_graph" à "True" permet de ne pas effacer le graphe de calcule.
r.backward(retain_graph=True)
r.backward()
print(repr(t.grad))  # On remarque alors que pytorch "accumule" les gradients au lieux d'écrire la nouvelle valuer par dessus l'acienne.


In [ ]:
# Utiliser un contexte "torch.no_grad()" permet de ne pas construire le graph de calcul, et donc d'économiser la mémoire, et d'accélérer le calcul
t = torch.arange(4, dtype=torch.float32, requires_grad=True)
with torch.no_grad():
    r = torch.sum(t**2)
print(r.requires_grad)

## III) Les Modules pytorch

Dans pytorch une brique de modèle (une couche d'un modèle feed forward, ...) comme un modèle entier dérivent de la classe Module.
Les classes filles doivent implémenter la fonction "forward" qui correspond à la passe avant de la couche/du modèle (aka: l'opération effectuée)

In [ ]:
lin = torch.nn.Linear(2, 3)  # certaines couches courrament utilisées sont déjà implémentées

In [ ]:
help(lin)  # La couche linéaire (y = x@A.T + b) attend en entrée un tenseur de shape (n_observations, features_in) et renvoit un tenseur de shape (n_observations, features_out)

In [ ]:
r = lin(torch.rand(10, 2))  # on peut appliquer l'opération correspondante en utilisant l'opérateur __call__
print(repr(r.shape))

In [ ]:
list(lin.parameters())  # on peut obtenir un itérable des paramètres d'un module ave cla méthode "parameters"

In [ ]:
from typing import Callable

class Layer(torch.nn.Module):
    """
    Un objet 'Layer' est une couche cachée d'un réseau feed forward
    """

    def __init__(self, in_features: int, out_features: int, activation: Callable, dropout: float):
        """
        Parameters
        ----------
        in_features : int
            number of features in input tensors
        out_features : int
            number of features in output
        activation : Callable
            activation function applied after linear projection and batch normalization
        dropout : float
            dropout probability for gidden layers
        
        Returns
        -------
        torch.Tensor :
            tensor of shape (N, H_out)
        """
        super().__init__()
        self.linear = torch.nn.Linear(in_features, out_features)
        self.batch_norm = torch.nn.BatchNorm1d(out_features)
        self.activation = activation
        self.dropout = torch.nn.Dropout(dropout)

    def forward(self, X: torch.Tensor) -> torch.Tensor:
        """
        Parameters
        ----------
        X : torch.Tensor
            tensor of shape (N, H_in)
        
        Returns
        -------
        torch.Tensor :
            tensor of shape (N, H_out)
        """
        X = self.linear(X)
        X = self.batch_norm(X)
        X = self.activation(X)
        X = self.dropout(X)
        return X

In [ ]:
class FeedForwardRegressor(torch.nn.Module):
    """
    Un objet 'FeedForwardRegressor' est un modèle feed forward pour la régression
    """

    def __init__(self, in_features: int, hidden_features: list[int], activation: Callable = F.relu, dropout: float = 0.):
        super().__init__()
        self.hidden_layers = torch.nn.ModuleList()
        for out_features in hidden_features:
            self.hidden_layers.append(Layer(in_features, out_features, activation, dropout))
            in_features = out_features
        self.output_projection = torch.nn.Linear(out_features, 1)
    
    def forward(self, X: torch.Tensor) -> torch.Tensor:
        """
        Parameters
        ----------
        X : torch.Tensor
            tensor of shape (N, H_in)
        
        Returns
        -------
        torch.Tensor :
            tensor of shape (N, H_out)
        """
        for layer in self.hidden_layers:
            X = layer(X)
        return self.output_projection(X)

In [ ]:
ffr = FeedForwardRegressor(2, [10, 10, 10])
ffr(torch.rand(10, 2))

In [ ]:
list(ffr.children())  # On peut récupérer un itérable sur les sous modules avec la méthode "children"

In [ ]:
len(list(ffr.parameters()))  # La méthode "parameter" renvoit aussi les groupes de paramètres des sous modules

In [ ]:
# Certains Modules ne doivent pas avoir le même comportement lors de l'entraînement et de l'évaluation.
# Un flag permet de déterminer si le module est en mode entraînement ou évaluation
dropout = torch.nn.Dropout(p=0.5)
dropout.training

In [ ]:
# on peut passer un module (et ses sous modules recursivement) en mode entraînement avec la méthode "train"
dropout.train()
dropout(torch.rand(3, 3))

In [ ]:
# on peut passer un module (et ses sous modules recursivement) en mode entraînement avec la méthode "eval"
dropout.eval()
dropout(torch.rand(3, 3))

## IV) La boucle d'entraînement

In [ ]:
def y(x: torch.Tensor) -> torch.Tensor:
    return x**2 + torch.normal(0., 0.1, x.shape)

In [ ]:
x_train = torch.linspace(-1., 1., 160).unsqueeze(1)
y_train = y(x_train)
x_val = torch.rand((40, 1)) * 2 - 1
y_val = y(x_val)
plt.scatter(x_train.reshape(-1).tolist(), y_train.reshape(-1).tolist(), label="training")
plt.scatter(x_val.reshape(-1).tolist(), y_val.reshape(-1).tolist(), label="validation")
plt.legend()

In [ ]:
def loss_function(y_pred: torch.Tensor, y_target: torch.Tensor) -> torch.Tensor:
    """
    returns the mean squared error loss

    Parameters
    ----------
    y_pred : torch.Tensor
        tensor of shape (N,)
    y_target : torch.Tensor
        tensor of shape (N,)
    
    Returns
    -------
    torch.Tensor :
        scalar tensor
    """
    return F.mse_loss(y_pred, y_target)

In [ ]:
def train_loop(model: torch.nn.Module, optimier: torch.optim.Optimizer, train_data: tuple[torch.Tensor], val_data: tuple[torch.Tensor], loss_function: Callable, n_steps: int = 100, patience: int = 10):
    """
    performs the training for a given number of iterations
    """
    x_train, y_train = train_data
    x_val, y_val = val_data
    best_metric = float("inf")
    best_step = 0
    checkpoint = deepcopy(model.state_dict())
    for step in range(n_steps):
        # setting the gradient to zero to avoid gradient accumulation
        optimizer.zero_grad()
        # training
        model.train()
        loss = loss_function(model(x_train), y_train)
        loss.backward()
        # validation
        model.eval()
        with torch.no_grad():
            metric = loss_function(model(x_val), y_val).item()
        if metric < best_metric:
            best_metric = metric
            best_step = step
            checkpoint = deepcopy(model.state_dict())
        elif step - best_step == patience:
            print("early stopping")
            break
        # printing progress
        print(f"Step {step}: loss = {loss.item():.3g}, metric = {metric:.3g}", flush=True)
        # step of the optimizer
        optimizer.step()
    model.load_state_dict(checkpoint)

In [ ]:
# Le namespace "optim" contient les différents algorithmes d'optimisation
ffr = FeedForwardRegressor(1, [30, 30])
optimizer = torch.optim.Adam(ffr.parameters(), lr=1.0E-3)
train_loop(ffr, optimizer, (x_train, y_train), (x_val, y_val), F.mse_loss, n_steps=200, patience=100)

In [ ]:
x_test = torch.linspace(-1, 1, 1000)
y_target = x_test**2
y_pred = ffr(x_test.unsqueeze(-1)).squeeze(-1)
plt.plot(x_test.tolist(), y_target.tolist(), color="k", label="target")
plt.plot(x_test.tolist(), y_pred.tolist(), color="r", label="prediction")
plt.legend()

## V) Une loss pour la classification

In [ ]:
def y(x: torch.Tensor) -> torch.Tensor:
    """
    returns the binary classification of the given (x1, x2) points

    Parameters
    ----------
    x : torch.Tensor
        tensor of shape (n, 2) of floats in [-1, 1]

    Returns
    -------
    torch.Tensor
        tensor of shape (n, 1) of type long, of values in {0, 1}
    """
    L = 0.8
    return (((x[:, 0] + x[:, 1]) % L) > L/2).float().unsqueeze(-1)


def generate_data(n: int) -> tuple[torch.Tensor]:
    """
    generate an (x, y) tuple of tensors of n random observations
    """
    x = torch.rand(n, 2) * 2 - 1
    y_ = y(x)
    return x, torch.where(torch.rand(n, 1) < 0.05, 1-y_, y_)

In [ ]:
x_train, y_train = generate_data(800)
x_val, y_val = generate_data(200)

In [ ]:
y_train.shape

In [ ]:
image = y(torch.stack(torch.meshgrid([torch.linspace(-1., 1., 500)]*2, indexing="ij"), dim=-1).reshape(-1, 2)).reshape(500, 500)
plt.imshow(image, extent=(-1, 1, -1, 1), origin="lower", cmap="Greys", vmin=0, vmax=1)
plt.scatter(x_train[:, 0], x_train[:, 1], c=[f"C{c.item()}" for c in y_train.int()], marker=".", label="train")
plt.scatter(x_val[:, 0], x_val[:, 1], c=[f"C{c.item()}" for c in y_val.int()], marker="+", label="validation")
plt.legend()

In [ ]:
class FeedForwardClassifier(torch.nn.Module):
    """
    Un objet 'FeedForwardRegressor' est un modèle feed forward pour la classification binaire
    """

    def __init__(self, in_features: int, hidden_layers: list[int], activation: Callable = F.relu, dropout: float = 0.):
        super().__init__()
        self.layers = torch.nn.ModuleList()
        for out_features in hidden_layers:
            self.layers.append(Layer(in_features, out_features, activation, dropout))
            in_features = out_features
        self.output_projection = torch.nn.Linear(out_features, 1)

    def forward(self, X: torch.Tensor) -> torch.Tensor:
        for layer in self.layers:
            X = layer(X)
        return torch.sigmoid(self.output_projection(X))

In [ ]:
ffc = FeedForwardClassifier(2, [30, 30, 30, 30])
optimizer = torch.optim.Adam(ffc.parameters(), lr=1.0E-3)
train_loop(ffc, optimizer, (x_train, y_train), (x_val, y_val), F.binary_cross_entropy, n_steps=1000, patience=100)

In [ ]:
with torch.no_grad():
    image = ffc(torch.stack(torch.meshgrid([torch.linspace(-1., 1., 500)]*2, indexing="ij"), dim=-1).reshape(-1, 2)).reshape(500, 500)

In [ ]:
plt.imshow(image.numpy(), origin="lower", cmap="Greys")

## VI) Exercice

In [1]:
import pandas as pd
pd.set_option('display.max_rows', 500)

In [2]:
df = pd.read_csv("../datasets/parcoursup_2021.csv")

In [6]:
df.columns

Index(['code_UAI', 'nom_etablissement', 'code_departement', 'nom_formation',
       'code_formation_parcoursup', 'type_contrat', 'formation_selective',
       'concour', 'nombre_candidats', 'taux_admission', 'taux_femmes',
       'taux_boursiers', 'taux_meme_academie', 'taux_meme_etablissement',
       'taux_bac_technologique', 'taux_bac_pro', 'taux_mention_assez_bien',
       'taux_mention_bien', 'taux_mention_tres_bien',
       'taux_mention_tres_bien_felicitations'],
      dtype='object')

In [17]:
df_eval = df.sample(frac=0.15)
df_other = df.drop(df_eval.index)
df_train = df_other.sample(frac=0.8235)
df_val = df_other.drop(df_train.index)
df_test = df_other.sample(frac=0.15)

In [15]:
df_train.to_csv("../datasets/train.csv", index=False)
df_test.to_csv("../datasets/test.csv", index=False)
df_eval.to_csv("../datasets/eval.csv", index=False)